<a href="https://colab.research.google.com/github/mustafabozkaya/Deep_Learning_Bootcamp/blob/master/AI/ML%20Challenge/eye_pictures_of_males_and_females_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download the images


We can use **GoogleDriveDownloader** form **google_drive_downloader** library in Python to download the shared files from the shared Google drive link: https://drive.google.com/file/d/1f7uslI-ZHidriQFZR966_aILjlkgDN76/view?usp=sharing

The file id in the above link is: **1f7uslI-ZHidriQFZR966_aILjlkgDN76**

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1f7uslI-ZHidriQFZR966_aILjlkgDN76',
                                    dest_path='content/eye_gender_data.zip',
                                    unzip=True)

Unzipping...

/usr/local/lib/python3.7/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1f7uslI-ZHidriQFZR966_aILjlkgDN76" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


We have all the files from the shared Google drive link downloaded in the colab environment.

## Loading Libraries
All Python capabilities are not loaded to our working environment by default (even they are already installed in your system). So, we import each and every library that we want to use.

We chose alias names for our libraries for the sake of our convenience (numpy --> np and pandas --> pd, tensorlow --> tf).

Note: You can import all the libraries that you think will be required or can import it as you go along.

In [ ]:
import pandas as pd                                     # Data analysis and manipultion tool
import numpy as np                                      # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf                                 # Deep Learning Tool
import os                                               # OS module in Python provides a way of using operating system dependent functionality
import cv2                                              # Library for image processing
from sklearn.model_selection import train_test_split    # For splitting the data into train and validation set
from sklearn.metrics import f1_score

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')
#!unzip ./content/eye_gender_data.zip
#!unzip ./drive/MyDrive/eye_gender_data.zip -d ./content/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading and preparing training data


In [ ]:
labels = pd.read_csv("./content/eye_gender_data/Training_set.csv")   # loading the labels
file_paths = [[fname, './content/eye_gender_data/train/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
train_data = pd.merge(images, labels, how = 'inner', on = 'filename')

train = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(train_data)):
  
  img_array = cv2.imread(train_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  train.append([new_img_array])

In [ ]:
x_train = np.array(train)
y_train = train_data.copy()['label'].values
x_train.shape, y_train.shape


((9220, 1, 100, 100), (9220,))

## Data Pre-processing
It is necessary to bring all the images in the same shape and size, also convert them to their pixel values because all machine learning or deep learning models accepts only the numerical data. Also we need to convert all the labels from categorical to numerical values.

In [ ]:
x_train = x_train / 255
x_train = x_train[:, 0, :, :]

In [ ]:
x_train.shape

y_train = y_train == 'male'

y_train = np.array(y_train)
y_train = y_train.astype(int)
y_train[:10]

array([1, 0, 0, 0, 1, 1, 0, 1, 0, 1])

## Building Model & Hyperparameter tuning
Now we are finally ready, and we can train the model.


In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt


def create_model():
  model = Sequential()
  model.add(Flatten(input_shape=(100, 100)))
  model.add(Dense(5000, activation='relu'))
  model.add(Dense(1000, activation='relu'))
  model.add(Dense(200, activation='relu'))
  model.add(Dense(2, activation='softmax'))
  return model

In [ ]:
model = create_model()
model.compile(optimizer='SGD', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
              )
model.fit(x_train, y_train, epochs=20, validation_split=0.2)

Epoch 1/20
231/231 [==============================] - 7s 20ms/step - loss: 0.6452 - accuracy: 0.6509 - val_loss: 0.5454 - val_accuracy: 0.7191
Epoch 2/20
231/231 [==============================] - 4s 18ms/step - loss: 0.5444 - accuracy: 0.7215 - val_loss: 0.4671 - val_accuracy: 0.8042
Epoch 3/20
231/231 [==============================] - 4s 19ms/step - loss: 0.4911 - accuracy: 0.7644 - val_loss: 0.4621 - val_accuracy: 0.7831
Epoch 4/20
231/231 [==============================] - 4s 19ms/step - loss: 0.4505 - accuracy: 0.7958 - val_loss: 0.5926 - val_accuracy: 0.7164
Epoch 5/20
231/231 [==============================] - 4s 18ms/step - loss: 0.4231 - accuracy: 0.8075 - val_loss: 0.4230 - val_accuracy: 0.8080
Epoch 6/20
231/231 [==============================] - 4s 19ms/step - loss: 0.4064 - accuracy: 0.8205 - val_loss: 0.5274 - val_accuracy: 0.7657
Epoch 7/20
231/231 [==============================] - 4s 19ms/step - loss: 0.4014 - accuracy: 0.8227 - val_loss: 0.3933 - val_accuracy: 0.8330

In [ ]:
labels = pd.read_csv("./content/eye_gender_data/Testing_set.csv")   # loading the labels
file_paths = [[fname, './content/eye_gender_data/test/' + fname] for fname in labels['filename']]
images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_data = pd.merge(images, labels, how = 'inner', on = 'filename')

test = []     # initialize an empty numpy array
image_size = 100      # image size taken is 100 here. one can take other size too
for i in range(len(test_data)):
  
  img_array = cv2.imread(test_data['filepaths'][i], cv2.IMREAD_GRAYSCALE)   # converting the image to gray scale

  new_img_array = cv2.resize(img_array, (image_size, image_size))      # resizing the image array
  test.append([new_img_array])

In [ ]:
x_test = np.array(test)
x_test = x_test / 255
x_test = x_test[:, 0, :, :]
x_test.shape

(2305, 100, 100)

## Validate the model


array([[0.02360322, 0.97639674],
       [0.503485  , 0.49651498],
       [0.13322322, 0.8667768 ],
       ...,
       [0.01208679, 0.9879132 ],
       [0.00839145, 0.99160856],
       [0.10736485, 0.8926352 ]], dtype=float32)

## Predict The Output For Testing Dataset 😅
We have trained our model, evaluated it and now finally we will predict the output/target for the testing data (i.e. Test.csv).

#### Load Test Set
Load the test data on which final submission is to be made.

## Data Pre-processing on test_data


In [ ]:
test_data['filename']

0          Image_1.jpg
1          Image_2.jpg
2          Image_3.jpg
3          Image_4.jpg
4          Image_5.jpg
             ...      
2300    Image_2301.jpg
2301    Image_2302.jpg
2302    Image_2303.jpg
2303    Image_2304.jpg
2304    Image_2305.jpg
Name: filename, Length: 2305, dtype: object

### Make Prediction on Test Dataset
Time to make a submission!!!

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred = [np.argmax(i) for i in y_pred]

## **How to save prediciton results locally via jupyter notebook?**
If you are working on Jupyter notebook, execute below block of codes. A file named 'submission.csv' will be created in your current working directory.

In [ ]:
res = pd.DataFrame({'filename': test_data['filename'], 'label': y_pred})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
#res.to_csv("submission.csv", index = False)      # the csv file will be saved locally on the same location where this notebook is located.

In [ ]:
res.loc[res["label"] == 1, "label"] = 'male'
res.loc[res["label"] == 0, "label"] = 'female'
finale_submission = res['label']
finale_submission.to_csv("finale_submission.csv", index = False) 

from google.colab import files        
files.download('finale_submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **OR,**
**If you are working on Google Colab then use the below set of code to save prediction results locally**

## **How to save prediction results locally via colab notebook?**
If you are working on Google Colab Notebook, execute below block of codes. A file named 'prediction_results' will be downloaded in your system.

In [ ]:
res = pd.DataFrame({'filename': test_data['filename'], 'label': y_pred})  # prediction is nothing but the final predictions of your model on input features of your new unseen test data
res.to_csv("submission.csv", index = False) 

# To download the csv file locally
from google.colab import files        
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Well Done! 👍**
You are all set to make a submission. Let's head to the **[challenge page](https://dphi.tech/challenges/4-week-deep-learning-online-bootcamp-final-assignment-sex-determination-by-morphometry-of-eyes/144/submit)** to make the submission.